# SNN (Nengo/NengoDL) su NSL-KDD — Colab

Notebook end-to-end: clona la repo, installa requisiti, scarica NSL-KDD, preprocessing, rete Nengo con neuroni LIF, training con NengoDL, valutazione e salvataggio parametri.

- Repo: https://github.com/devedale/snn-ids
- Framework: Nengo + NengoDL (TensorFlow backend)
- Codifica: rate coding (ripetizione su T step)
- Task: binaria (anomalo vs normale)


In [ ]:
# Clona la repo e installa requisiti
!rm -rf /content/snn-ids && git clone https://github.com/devedale/snn-ids.git /content/snn-ids -q
%cd /content/snn-ids
!pip -q install -r requirements.txt nengo nengo-dl scikit-learn pyyaml



In [ ]:
# Installazione dipendenze Nengo/NengoDL
!pip -q install nengo nengo-dl pandas scikit-learn pyyaml
import numpy as np, pandas as pd
from pathlib import Path


In [ ]:
# Download NSL-KDD (train/test) e conversione CSV→dataset SNN via repo
base = Path('/content/data'); base.mkdir(parents=True, exist_ok=True)
urls = {
    'KDDTrain+': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt',
    'KDDTest+':  'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt',
}
for name, url in urls.items():
    !wget -q -O /content/data/{name}.txt "{url}"
!python -m tools.csv_to_snn_dataset --input /content/data/KDDTrain+.txt --output /content/data/nslkdd_snn_train.csv --label-col label
!python -m tools.csv_to_snn_dataset --input /content/data/KDDTest+.txt --output /content/data/nslkdd_snn_test.csv --label-col label
!ls -lh /content/data


In [ ]:
# Carica dataset SNN già convertiti (train/test) e label reali dai raw
train_df = pd.read_csv('/content/data/nslkdd_snn_train.csv')
test_df  = pd.read_csv('/content/data/nslkdd_snn_test.csv')

# Label da file raw
cols = [
  "duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
  "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
  "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
  "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
  "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
  "dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
  "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"
]
raw_train = pd.read_csv('/content/data/KDDTrain+.txt', names=cols)
raw_test  = pd.read_csv('/content/data/KDDTest+.txt',  names=cols)
y_train = (raw_train['label'].astype(str).str.lower() != 'normal').astype(int).values
y_test  = (raw_test['label'].astype(str).str.lower()  != 'normal').astype(int).values

# Estrai features
feat_cols = [c for c in train_df.columns if c != 'timestamp']
X_train = train_df[feat_cols].values.astype(np.float32)
X_test  = test_df[feat_cols].values.astype(np.float32)
input_size, output_size = X_train.shape[1], 2
X_train.shape, X_test.shape


In [ ]:
# Costruzione rete Nengo e training con NengoDL
import nengo, nengo_dl, tensorflow as tf

timesteps = 10
with nengo.Network() as net:
  inp = nengo.Node(np.zeros(input_size))
  ens1 = nengo.Ensemble(n_neurons=256, dimensions=input_size, neuron_type=nengo.LIF())
  nengo.Connection(inp, ens1)
  ens2 = nengo.Ensemble(n_neurons=output_size, dimensions=output_size, neuron_type=nengo.LIF())
  nengo.Connection(ens1, ens2)
  p_out = nengo.Probe(ens2, synapse=0.1)

# rate coding per NengoDL: ripeti features su T
X_train_enc = np.repeat(X_train[:, None, :], timesteps, axis=1)
X_test_enc  = np.repeat(X_test[:,  None, :], timesteps, axis=1)
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=output_size)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=output_size)
y_train_enc = np.repeat(y_train_oh[:, None, :], timesteps, axis=1)
y_test_enc  = np.repeat(y_test_oh[:,  None, :], timesteps, axis=1)

minibatch = 128
with nengo_dl.Simulator(net, minibatch_size=minibatch) as sim:
  # input mapping: primo input del modello → X
  inputs = {inp: X_train_enc}
  targets = {p_out: y_train_enc}
  sim.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.MSE)
  sim.fit(inputs, targets, epochs=5, validation_split=0.2, verbose=2)
  # valutazione
  test_dict = {inp: X_test_enc}
  sim.run_steps(X_test_enc.shape[1], data=test_dict)
  out = sim.data[p_out]

pred = out.mean(axis=1)
acc = (pred.argmax(axis=1) == y_test).mean()
print('Test accuracy:', acc)



In [ ]:
# Salvataggio artefatti (parametri rete)
import pickle, os
os.makedirs('/content/out', exist_ok=True)
with open('/content/out/nengo_nslkdd.pkl', 'wb') as f:
  pickle.dump(net, f)
print('Salvato /content/out/nengo_nslkdd.pkl')
